In [1]:

%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import imutils
import glob
import cv2
import shutil
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 500)
tqdm.pandas()

In [3]:
!pip install tqdm

In [3]:
## Loading Haar Cascade
## Taken from https://github.com/opencv/opencv/tree/master/data/haarcascades
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [10]:
## Defining the function to extract the face 
def face_extractor(origin, destination, fc):
    ## Importing image using open cv
    img = cv2.imread(origin,1)

    ## Resizing to constant width
    img = imutils.resize(img, width=200)
    
    ## Finding actual size of image
    H,W,_ = img.shape
    
    ## Converting BGR to RGB
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    ## Detecting faces on the image
    face_coord = fc.detectMultiScale(gray,1.2,10,minSize=(50,50))
    
    ## If only one face is found
    if len(face_coord) == 1:
        X, Y, w, h = face_coord[0]
    
    ## If no face found --> SKIP
    elif len(face_coord)==0:
        return None
    
    ## If multiple faces are found take the one with largest area
    else:
        max_val = 0
        max_idx = 0
        for idx in range(len(face_coord)):
            _, _, w_i, h_i = face_coord[idx]
            if w_i*h_i > max_val:
                max_idx = idx
                max_val = w_i*h_i
            else:
                pass
            
            X, Y, w, h = face_coord[max_idx]
    
    ## Crop and export the image
    img_cp = img[
            max(0,Y - int(0.35*h)): min(Y + int(1.35*h), H),
            max(0,X - int(w*0.35)): min(X + int(1.35*w), W)
        ].copy()
    cv2.imwrite(destination, img_cp)

In [11]:
## Defining destination path
path = r'D:/tutors_point/project_work/data/faces/'

## Finding all the images in the folder
item_list = glob.glob('./data/first_50000/*.jpg')
print(len(item_list))

1959


In [13]:
for org in tqdm(item_list):
    image=cv2.imread(org);
    if type(image) is np.ndarray:
        face_extractor(origin = org, destination = path+org.split('\\')[-1], fc=face_cascade)

  0%|          | 0/1959 [00:00<?, ?it/s]

In [46]:
## Finding all the images and separating in training and validation
item_list = glob.glob(path+'*.jpg')
for idx,item in zip(tqdm(range(1,1180)),item_list):
    if idx <= 700:
        destination = 'training/'
    else:
        destination = 'validation/'
        shutil.move(
            item.split('\\')[0]+'/'+item.split('\\')[1], 
            item.split('\\')[0]+'/'+destination+item.split('\\')[1]
        )

  0%|          | 0/1179 [00:00<?, ?it/s]